# Artificial Intel & Heurist Prg - Project 3

In [9]:
# Hexapawn Board Template
# Simple and smart starting point for a 3x3 hexapawn game board.

class HexapawnBoard:
    def __init__(self):
        # "w" = white pawn, "b" = black pawn, "." = empty
        self.board = [
            ["b", "b", "b"],
            [".", ".", "."],
            ["w", "w", "w"],
        ]

    def display(self):
        for row in self.board:
            print(" ".join(row))
        print()

    def clone(self):
        # Helpful for AI search trees
        new_board = HexapawnBoard()
        new_board.board = [row[:] for row in self.board]
        return new_board

    def get_legal_moves(self, player):
        # Placeholder for move logic
        # player = "w" or "b"
        moves = []
        # TODO: Implement legal move generation
        return moves

    def make_move(self, move):
        # Placeholder: apply (from_row, from_col, to_row, to_col)
        (fr, fc, tr, tc) = move
        self.board[tr][tc] = self.board[fr][fc]
        self.board[fr][fc] = "."

if __name__ == "__main__":
    b = HexapawnBoard()
    b.display()

    print("Test Move", "\n")
    b.make_move((2, 1, 1, 1))
    b.display()


b b b
. . .
w w w

Test Move 

b b b
. w .
w . w

